# 3.3 Concise LR Implementation
* Implement LR using torch machinery 
* This gives the basic loop along with mini_batching which can be copied.

In [3]:
import numpy as np
import torch
from torch.utils import data

In [4]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, w.shape[0]))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1,1)) 

In [6]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

#### NOTE:
The following is a useful way of creating data iterator.

In [11]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [15]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

### Training Model

#### NOTE:
Accessing network parameters and initization

In [16]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [17]:
loss = nn.MSELoss()

#### NOTE: 
`net.parameters()` gives access to the parameters of the model. 

In [22]:
[xx for xx in net.parameters()]

[Parameter containing:
 tensor([[ 0.0113, -0.0019]], requires_grad=True),
 Parameter containing:
 tensor([0.], requires_grad=True)]

In [19]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [24]:
# training loop
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000188
epoch 2, loss 0.000091
epoch 3, loss 0.000091


### Display Learned Weights

In [25]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([0.0002, 0.0006])
error in estimating b: tensor([-0.0005])
